In [1]:
import pandas as pd

In [2]:
premium = pd.DataFrame(
    {"premium": [5812, 4908, 5454, 5165, 5214, 5230, 4992, 5466, 5226, 4962]},
    index=range(1988, 1998),
)
premium.index.name = "ay"
premium.to_parquet("premium.parquet")
premium

,premium
ay,
1988,5812
1989,4908
1990,5454
1991,5165
1992,5214
1993,5230
1994,4992
1995,5466
1996,5226


In [3]:
pd_loss = """1988 952 1529 2813 3647 3724 3832 3899 3907 3911 3912 1997
1989 849 1564 2202 2432 2468 2487 2513 2526 2531 2527 1998
1990 983 2211 2830 3832 4039 4065 4102 4155 4268 4274 1999
1991 1657 2685 3169 3600 3900 4320 4332 4338 4341 4341 2000
1992 932 1940 2626 3332 3368 3491 3531 3540 3540 3583 2001
1993 1162 2402 2799 2996 3034 3042 3230 3238 3241 3268 2002
1994 1478 2980 3945 4714 5462 5680 5682 5683 5684 5684 2003
1995 1240 2080 2607 3080 3678 2004 4117 4125 4128 4128 1997
1996 1326 2412 3367 3843 3965 4127 4133 4141 4142 4144 2005
1997 1413 2683 3173 3674 3805 4005 4020 4095 4132 4139 2006"""

paid_loss = pd_loss.split("\n")
paid_loss = [x.split() for x in paid_loss]
cols = ["ay"] + [str(x) for x in range(1, 11)] + ["source"]
paid_loss = pd.DataFrame(paid_loss, columns=cols)
paid_loss = paid_loss.set_index("ay")

paid_loss[["source"]].to_parquet("paid_loss_source.parquet")

paid_loss = paid_loss.drop(columns=["source"])
paid_loss.to_parquet("paid_loss.parquet")

paid_loss

,1,2,3,4,5,6,7,8,9,10
ay,,,,,,,,,,
1988,952,1529,2813,3647,3724,3832,3899,3907,3911,3912
1989,849,1564,2202,2432,2468,2487,2513,2526,2531,2527
1990,983,2211,2830,3832,4039,4065,4102,4155,4268,4274
1991,1657,2685,3169,3600,3900,4320,4332,4338,4341,4341
1992,932,1940,2626,3332,3368,3491,3531,3540,3540,3583
1993,1162,2402,2799,2996,3034,3042,3230,3238,3241,3268
1994,1478,2980,3945,4714,5462,5680,5682,5683,5684,5684
1995,1240,2080,2607,3080,3678,2004,4117,4125,4128,4128
1996,1326,2412,3367,3843,3965,4127,4133,4141,4142,4144


In [4]:
rptloss = """1988 1722 3830 3603 3835 3873 3895 3918 3918 3917 3917 1997
1989 1581 2192 2528 2533 2528 2530 2534 2541 2538 2532 1998
1990 1834 3009 3488 4000 4105 4087 4112 4170 4271 4279 1999
1991 2305 3473 3713 4018 4295 4334 4343 4340 4342 4341 2000
1992 1832 2625 3086 3493 3521 3563 3542 3541 3541 3587 2001
1993 2289 3160 3154 3204 3190 3206 3351 3289 3267 3268 2002
1994 2881 4254 4841 5176 5551 5689 5683 5688 5684 5684 2003
1995 2489 2956 3382 3755 4148 4123 4126 4127 4128 4128 2004
1996 2541 3307 3789 3973 4031 4157 4143 4142 4144 4144 2005
1997 2203 2934 3608 3977 4040 4121 4147 4155 4183 4181 2006"""

rpt_loss = rptloss.split("\n")
rpt_loss = [x.split() for x in rpt_loss]
cols = ["ay"] + [str(x) for x in range(1, 11)] + ["source"]
rpt_loss = pd.DataFrame(rpt_loss, columns=cols)
rpt_loss = rpt_loss.set_index("ay")

rpt_loss[["source"]].to_parquet("rpt_loss_source.parquet")

rpt_loss = rpt_loss.drop(columns=["source"])
rpt_loss.to_parquet("rpt_loss.parquet")

rpt_loss

,1,2,3,4,5,6,7,8,9,10
ay,,,,,,,,,,
1988,1722,3830,3603,3835,3873,3895,3918,3918,3917,3917
1989,1581,2192,2528,2533,2528,2530,2534,2541,2538,2532
1990,1834,3009,3488,4000,4105,4087,4112,4170,4271,4279
1991,2305,3473,3713,4018,4295,4334,4343,4340,4342,4341
1992,1832,2625,3086,3493,3521,3563,3542,3541,3541,3587
1993,2289,3160,3154,3204,3190,3206,3351,3289,3267,3268
1994,2881,4254,4841,5176,5551,5689,5683,5688,5684,5684
1995,2489,2956,3382,3755,4148,4123,4126,4127,4128,4128
1996,2541,3307,3789,3973,4031,4157,4143,4142,4144,4144


In [5]:
loss_data = (
    rpt_loss.reset_index()
    .melt(var_name="dev", value_name="rpt_loss", id_vars=["ay"])
    .rename(columns={"ay": "acc"})
    .assign(dev=lambda x: x["dev"].astype(int))
    .assign(acc=lambda x: x["acc"].astype(int))
    .merge(premium.reset_index(), how="left", left_on="acc", right_on="ay")
    .drop(columns=["ay"])
    .merge(
        paid_loss.reset_index()
        .melt(var_name="dev", value_name="paid_loss", id_vars=["ay"])
        .rename(columns={"ay": "acc"})
        .assign(dev=lambda x: x["dev"].astype(int))
        .assign(acc=lambda x: x["acc"].astype(int)),
        how="left",
        on=["acc", "dev"],
    )
)[["acc", "dev", "premium", "rpt_loss", "paid_loss"]]
loss_data.head()

,acc,dev,premium,rpt_loss,paid_loss
0,1988,1,5812,1722,952
1,1989,1,4908,1581,849
2,1990,1,5454,1834,983
3,1991,1,5165,2305,1657
4,1992,1,5214,1832,932
